In [1]:
import sys
sys.path.append("./Incontext-learning") # this part works for goole colab

In [5]:
import transformers
import torch

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Task
Modify the provided code to perform a classification task instead of a causal language modeling task. This involves loading a suitable classification model, preparing a classification dataset, fine-tuning the model on the dataset, and evaluating its performance.

## Load a classification model

### Subtask:
Instead of loading a causal language model, load a model specifically designed for sequence classification, such as `AutoModelForSequenceClassification`.


**Reasoning**:
The subtask is to load a sequence classification model. This requires importing the appropriate class and loading the model and tokenizer.



In [8]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "Qwen/Qwen3-0.6B" # Assuming a sequence classification version exists

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set the padding token to the EOS token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


# Load the sequence classification model
# Note: You might need to specify `num_labels` based on your classification task
model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels = 2).to(device)

print("Sequence classification model and tokenizer loaded successfully.")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

Some weights of Qwen3ForSequenceClassification were not initialized from the model checkpoint at Qwen/Qwen3-0.6B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Sequence classification model and tokenizer loaded successfully.


In [10]:
# Check current tokens
print("EOS token:", tokenizer.eos_token, tokenizer.eos_token_id)
print("PAD token:", tokenizer.pad_token, tokenizer.pad_token_id)

EOS token: <|im_end|> 151645
PAD token: <|endoftext|> 151643


## Prepare data

### Subtask:
Load and preprocess your classification dataset. This includes tokenizing the text and formatting the labels.


**Reasoning**:
Load a suitable classification dataset, define a preprocessing function to tokenize the text and prepare labels, and apply the function to the dataset.



In [11]:
from datasets import load_dataset

# Load a suitable classification dataset (e.g., SST-2 from GLUE)
dataset = load_dataset("glue", "sst2", split="validation")

# Define a preprocessing function
def preprocess_function(examples):
    # Tokenize the text
    tokenized_inputs = tokenizer(examples["sentence"], truncation=True)
    # Prepare the labels
    tokenized_inputs["labels"] = examples["label"]
    return tokenized_inputs

# Apply the preprocessing function to the dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Split into training and evaluation sets
# train_dataset = tokenized_dataset["validation"]
# eval_dataset = tokenized_dataset["validation"]
train_dataset = tokenized_dataset.select(range(100))
eval_dataset = tokenized_dataset.select(range(100, 200))

print("Dataset loaded, preprocessed, and split successfully.")
print("Training dataset example:", train_dataset[0])
print("Evaluation dataset example:", eval_dataset[0])

README.md: 0.00B [00:00, ?B/s]

sst2/train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

sst2/validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

sst2/test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Dataset loaded, preprocessed, and split successfully.
Training dataset example: {'sentence': "it 's a charming and often affecting journey . ", 'label': 1, 'idx': 0, 'input_ids': [275, 364, 82, 264, 34409, 323, 3545, 27887, 11618, 659, 220], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': 1}
Evaluation dataset example: {'sentence': 'neither parker nor donovan is a typical romantic lead , but they bring a fresh , quirky charm to the formula . ', 'label': 1, 'idx': 100, 'input_ids': [811, 2485, 6118, 261, 6329, 1513, 42657, 374, 264, 14260, 23467, 2990, 1154, 714, 807, 4446, 264, 7722, 1154, 67132, 31253, 311, 279, 14806, 659, 220], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': 1}


In [12]:
eval_dataset

Dataset({
    features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 100
})

## Fine-tune the model

### Subtask:
Train the classification model on your prepared dataset.


**Reasoning**:
Import the necessary classes and define the training arguments, then initialize and start the training process.



**Reasoning**:
The error indicates that `evaluation_strategy` is not a valid argument for `TrainingArguments`. Looking at the transformers documentation, `evaluation_strategy` was deprecated and replaced by `eval_strategy`. I will fix the typo and rerun the code.



In [14]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",  # Output directory
    eval_strategy="epoch",  # Evaluate every epoch
    learning_rate=2e-5,  # Learning rate
    per_device_train_batch_size=8,  # Batch size for training
    per_device_eval_batch_size=8,  # Batch size for evaluation
    num_train_epochs=1,  # Number of training epochs.  || in order to make it the same as in context learning
    weight_decay=0.01,  # Weight decay
    logging_dir="./logs",  # Directory for logs
    logging_steps=10, # Log every 10 steps
    report_to="none", # Disable Weights & Biases
    save_strategy='no'
)

# Ensure the tokenizer has the correct padding token and ID for the data collator
if tokenizer.pad_token is None:
  tokenizer.pad_token = tokenizer.eos_token
  tokenizer.pad_token_id = tokenizer.eos_token_id
if model.config.pad_token_id is None:
  model.config.pad_token_id = tokenizer.pad_token_id


# Initialize the data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Initialize the Trainer
trainer = Trainer(
    model=model,  # the loaded sequence classification model
    args=training_args,  # training arguments
    train_dataset=train_dataset,  # training dataset
    eval_dataset=eval_dataset,  # evaluation dataset
    data_collator=data_collator, # Use the data collator
)

# Start training
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.686700,2.691015
2,0.000300,1.706526
3,0.000000,1.843942


TrainOutput(global_step=39, training_loss=0.17617133679119099, metrics={'train_runtime': 21.623, 'train_samples_per_second': 13.874, 'train_steps_per_second': 1.804, 'total_flos': 28722135416832.0, 'train_loss': 0.17617133679119099, 'epoch': 3.0})